### This function renames files in a directory and if specied will resize the images
####  rename (source_dir, snum, new_ext, resize) where:
#### source_dir is the full path to the directory containing the image files to be processed
#### snum is an integer. If set to 0 the file names will not be changes but the files can be still be converted
#### to a new image format and also be resized
#### if snum is a non 0 integer the files are renamed in numerical sequence starting with the value of snum.
#### note file are renumber with "zeros" padding so that files are processedby python functions based on
#### their numerical order.
#### new_ext is a string specifying the new image format the files should be converted to.
#### note only these extensions are allowed 'jpg', 'jpe', 'jpeg', 'png', 'bmp', 'tiff'
#### if next_ext='same' the files will be kept in their original format
#### resize specifies the new size for the images. If resize='same' the images are new resized
#### if not set to 'same' resize must be a tuple of the form (height,width) where height and width
#### are integers.
#### To use this function cv2 and tqdm must be installed in your working environment
#### Note only files of the type listed under extensions above can be processed.
#### if a file in the source_dir is not of the type that can be processed by cv2.imread, an exception
#### occurs and a message is printed saying the file has been deleted from the directory.

In [1]:
import os
import cv2
from tqdm import tqdm

In [2]:
def rename (source_dir, snum, new_ext, resize):
    ext_list=['jpg', 'jpe', 'jpeg', 'png', 'bmp', 'tiff']
    if new_ext not in ext_list and new_ext !='same': # make sure new extension if specified is in acceptable list
        msg='\nthe new extension you specified {0} is not in the list of acceptable extensions '
        print(msg.format(new_ext))
        print('The list of valid extensions is ', ext_list, ' **** program terminated ****')
        return
    if resize !='same':
        if type(resize) is not tuple:
            msg='\ the entry for resize {0} is not a proper tuple '
            print(msg.format(resize))
            print('resize must either same or of the form (width, height)  where width and height are integer **** program terminated ****')
            return
    f_list=[]
    if new_ext !='same':
        new_ext=new_ext.lower()
    source_list=os.listdir(source_dir) 
    for f in source_list:
        f_path=os.path.join(source_dir, f)
        if os.path.isdir(f_path)==False:
            f_list.append(f)  # list of file names in source directory
    fc=len(f_list)  # determine number of d files to process  
    pad=0
    mod = 10     
    for i in range(1, fc + 1): # skip i=0 because 0 modulo anything is 0 and we don't want to increment pad
        if i % mod == 0:
            pad=pad+1                    
            mod =mod * 10
    good=0
    i=0
    for i in tqdm(range(fc)):
        f=f_list[i] 
        f_path=os.path.join(source_dir,f) #full path to the file
        filename=os.path.basename(f_path)
        index=f_path.rfind('.') # find location of last . in file name
        fname=f_path[:index]    # name of file - no extension
        ext=f_path[index + 1:]   # this does not include the period
        if ext=='jfif': # jfif files are the same as jpg files so change the extension to jpg
            ext='jpg'
        if snum !=0: # check if need to rename file
            fnew= str(i + snum).zfill(pad+1)  # rename the files name with leading zeros
        else:
            fnew=fname # use original file name
        new_path=os.path.join(source_dir, fnew) 
        if new_ext != 'same': # check if files will have a new extension
            fnew=fnew +'.' + new_ext 
        else:
            fnew=fnew + '.' + ext  # use old extension
        new_path= os.path.join (source_dir, fnew) # full path for the newly renamed file        
        try:
            img=cv2.imread(f_path) 
            dummy=img.shape
            if resize !='same':
                img=cv2.resize(img, resize) 
            os.remove(f_path)
            cv2.imwrite(new_path, img)
            good=good + 1
        except:
            print ('image ' ,filename, ' is an invalid image and will be deleted')
            os.remove(f_path)
    if new_ext=='same':            
        msg=' {0} files were renamed and saved with the original extensions'.format(good)
    else:
         msg='{0} files were renamed and saved with extension {1}'.format(good, new_ext) 
    print (msg)
    return 


In [3]:
source_dir=r'C:\Temp\BIRDS\predictor test set'
snum=1
new_ext='bmp'
resize='same'
rename (source_dir, snum, new_ext, resize )   

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 152.68it/s]

20 files were renamed and saved with extension bmp
